<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [3]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

Found 29031 files belonging to 7 classes.
Found 7378 files belonging to 7 classes.


In [5]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [6]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [7]:
test_batch

(array([[[[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         ...,
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.

In [8]:
len(test_batch)

2

In [9]:
train_batch[0].shape

(32, 256, 256, 3)

In [10]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [11]:
scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
test_batch[0].max()

1.0

In [12]:
len(train_data)

908

In [13]:
train_size = int(len(train_data)*0.8)
val_size = int(len(train_data)*0.2)+1
test_size = len(test_data)
print(train_size , val_size)

726 182


In [14]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [15]:
print(len(train),len(val),len(test))

726 182 231


## Step 3 - Model

In [44]:
model = Sequential(
    [
        Conv2D(32, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),

        Conv2D(64, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),

        Conv2D(32, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [49]:
model.compile(
    optimizer = 'adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [50]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 60, 60, 32)        18464     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 30, 32)      

In [51]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [ ]:
history = model.fit(train, validation_data = val, epochs = 20, callbacks = [tensorboard_callback])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


726/726 [==============================] - 139s 189ms/step - loss: 1.7164 - accuracy: 0.3177 - val_loss: 1.5972 - val_accuracy: 0.3821
Epoch 2/20
699/726 [===========================>..] - ETA: 2s - loss: 1.4335 - accuracy: 0.4489

In [ ]:
model.save('/content/gdrive/MyDrive/Human emotions/Saved_model')
